In [1]:
import json
from collections import OrderedDict
import os

building_name = 'ebu3b'
with open("metadata/%s_label_dict.json"%building_name, 'r') as fp:
    label_dict = json.load(fp)
with open("metadata/%s_sentence_dict.json"%building_name, "r") as fp:
    sentence_dict = OrderedDict(json.load(fp))

In [2]:
with open("metadata/%s_mallet_learning_sample.txt"%building_name, "w") as fp:
    for srcid, labels in label_dict.items():
        sentence = sentence_dict[srcid]
        sentence = ["newline" if word=="\n" else word for word in sentence]
        #print "EOB"
        assert(len(sentence)==len(labels))
        for i, (word,label) in enumerate(zip(sentence,labels)):
            if i==0:
                feat_prev_word= "prev:BOD"
            else:
                feat_prev_word = "prev:%s"%sentence[i-1].lower()
            feat_curr_word = "curr:%s"%word.lower()
            if i==len(sentence)-1:
                feat_next_word = "next:EOD"
            else:
                feat_next_word = "next:%s"%sentence[i+1].lower()
            #feat_curr_label = "curr_label:%s"%label
            fp.write("{0} {1} {2} {3}\n".format(feat_prev_word, feat_curr_word, feat_next_word, label))

In [3]:
os.system('java -cp "/home/jbkoh/tools/mallet-2.0.8/class:/home/jbkoh/tools/mallet-2.0.8/lib/mallet-deps.jar" cc.mallet.fst.SimpleTagger --train true --model-file model/{0}crf --threads 16 metadata/{0}_mallet_learning_sample.txt'.format(building_name))

0

In [4]:
with open('metadata/%s_mallet_test_sample.txt'%building_name, "w") as fp:
    for srcid, sentence in sentence_dict.items():
        sentence = ["newline" if word=="\n" else word for word in sentence]
        for i, word in enumerate(sentence):
            if i==0:
                feat_prev_word= "prev:BOD"
            else:
                feat_prev_word = "prev:%s"%sentence[i-1].lower()
            feat_curr_word = "curr:%s"%word.lower()
            if i==len(sentence)-1:
                feat_next_word = "next:EOD"
            else:
                feat_next_word = "next:%s"%sentence[i+1].lower()
            #feat_curr_label = "curr_label:%s"%label
            fp.write("{0} {1} {2}\n".format(feat_prev_word, feat_curr_word, feat_next_word))

In [11]:
os.system('java -Xmx12g -cp "/home/jbkoh/tools/mallet-2.0.8/class:/home/jbkoh/tools/mallet-2.0.8/lib/mallet-deps.jar" cc.mallet.fst.SimpleTagger --model-file model/{0}crf --threads 8 metadata/{0}_mallet_test_sample.txt > test_result/{0}_test.txt'.format(building_name))

0

In [12]:
with open('test_result/%s_test.txt'%building_name, 'r') as fp:
    word_label_dict = dict()
    for srcid, sentence in sentence_dict.items():
        word_label_list = list()
        for word in sentence:
            label = fp.readline()
            word_label_list.append((word, label))
        word_label_dict[srcid] = word_label_list

In [29]:
with open('test_result/%s_with_tags.txt'%building_name, 'w') as fp:
    for srcid, word_label_list in word_label_dict.items():
        fp.write("========= {0} =========\n".format(srcid))
        for (word, label) in word_label_list:
            fp.write('{:20s} {:20s} \n'.format(word, label))
        fp.write("===============================\n")

In [14]:
word_label_dict.items()[0]

(u'505_1_3014679',
 [(u'nae', 'none \n'),
  (u'05', 'identifier \n'),
  (u'n', 'none \n'),
  (u'2', 'identifier \n'),
  (u'1', 'identifier \n'),
  (u'dx', 'network_adapter \n'),
  (u'12', 'identifier \n'),
  (u'file', 'none \n'),
  (u'server', 'server \n'),
  (u'chw', 'chilled_water \n'),
  (u'system', 'system \n'),
  (u'fcwsp', 'none \n'),
  (u'sp', 'setpoint \n'),
  (u'newline', 'none \n'),
  (u'ebu', 'building \n'),
  (u'3', 'identifier \n'),
  (u'b', 'identifier \n'),
  (u'.', 'none \n'),
  (u'fschw', 'chilled_water \n'),
  (u'.', 'none \n'),
  (u'fcwdp', 'none \n'),
  (u'-', 'none \n'),
  (u'stpt', 'setpoint \n'),
  (u'newline', 'none \n'),
  (u'chilled', 'chilled \n'),
  (u'water', 'water \n'),
  (u'pressure', 'humidity \n'),
  (u'setpoint', 'setpoint \n'),
  (u'newline', 'none \n')])